<div style="background: linear-gradient(to bottom, #8e44ad, #9b59b6); padding: 20px; border-radius: 15px; text-align: center; font-family: 'Poppins', sans-serif; color: #ffffff; box-shadow: 0 8px 20px rgba(0, 0, 0, 0.2);">
    <h1 style="font-size: 3em; font-weight: bold; color: #ffffff; text-shadow: 2px 2px 8px rgba(0, 0, 0, 0.5);">Live Your Music ! </h1>
    <hr style="border: 2px solid rgba(255, 255, 255, 0.8); width: 75%; margin: 15px auto;">
    <h2 style="font-size: 2em; font-weight: 500; color: #ffffff; text-shadow: 1px 1px 5px rgba(0, 0, 0, 0.3);">Webscrapping Project</h2>
    <div style="font-size: 1.4em; margin-top: 15px;">
        <p><strong>Team Members:</strong> EL HACHEM Gabriel, FALCK Jade, LE QUERNEC Loévan</p>
        <p><strong>Class:</strong> DIA2</p>
    </div>
    <div style="margin-top: 20px; display: flex; justify-content: center; align-items: center; gap: 8px;">
        <div style="background: #ffffff; width: 15px; height: 15px; border-radius: 50%;"></div>
        <div style="background: rgba(255, 255, 255, 0.8); width: 12px; height: 12px; border-radius: 50%;"></div>
        <div style="background: #ffffff; width: 15px; height: 15px; border-radius: 50%;"></div>
    </div>
</div>

## Scraping via API

We will start by exploring how to interact with the Spotify Web API to retrieve data that enhances our music recommendation system. The Spotify API allows developers to access a wealth of musical data, including information on tracks, artists, albums, playlists, and genres. By leveraging this API, we can build personalized recommendations for tourists based on their music preferences and the country they are visiting.

In this notebook, we will implement various functions to interact with the API, enabling us to:

* Fetch popular tracks and playlists for specific countries.
* Retrieve recommendations tailored to user preferences.
* Explore genres, artists, and new releases.

Let's import the libraries necessary to use the spotify API.

In [25]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

Initalisation of the API keys and the Spotify Client.

In [26]:
# Vos identifiants API Spotify
CLIENT_ID = '05da08c935744445bec4905d0294bf84'
CLIENT_SECRET = 'ef136321382f47258017738c97f7c219'


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))

Hre is a function to search a specific by writing his name. Its allows to get informations about an artis, and specially its ID.

In [31]:
def search_artist(artist_name, limit=5):
    """
    Recherche des artistes correspondant à un nom donné.
    
    :param artist_name: Nom de l'artiste recherché.
    :param limit: Nombre maximum de résultats.
    """
    try:
        results = sp.search(q=f"artist:{artist_name}", type='artist', limit=limit)
        artists = [
            {
                'name': artist['name'],
                'id': artist['id'],
                'genres': artist['genres'],
                'popularity': artist['popularity'],
                'followers': artist['followers']['total'],
                'url': artist['external_urls']['spotify']
            }
            for artist in results['artists']['items']
        ]
        id = ""
        for artist in artists : 
            if artist['name'] == artist_name : 
                id = artist['id']
        return artists
    except Exception as e:
        print(f"Erreur lors de la recherche de l'artiste : {e}")
        return []

In [32]:
artist_id = search_artist("Jespfur")
artist_id

[{'name': 'Jespfur',
  'id': '7uPZzaOXKtwltRGEgeQN0r',
  'genres': [],
  'popularity': 36,
  'followers': 6172,
  'url': 'https://open.spotify.com/artist/7uPZzaOXKtwltRGEgeQN0r'}]

With this ID, we can now get the top tracks of a specific artist !

In [29]:
def get_artist_top_tracks(artist_id, country='FR'):
    """
    Récupère les morceaux les plus populaires d'un artiste.
    
    :param artist_id: ID de l'artiste.
    :param country: Code du pays (par défaut : 'FR').
    """
    try:
        tracks = sp.artist_top_tracks(artist_id, country=country)
        return [
            {
                'name': track['name'],
                'url': track['external_urls']['spotify'],
                'preview_url': track['preview_url']
            }
            for track in tracks['tracks']
        ]
    except Exception as e:
        print(f"Erreur lors de la récupération des morceaux populaires : {e}")
        return []

In [30]:
artist_top_tracks = get_artist_top_tracks(artist_id)
artist_top_tracks

[{'name': 'Comin Home',
  'url': 'https://open.spotify.com/track/1RAiHhh4HfUgcA1ATlg9yA',
  'preview_url': None},
 {'name': 'Condition',
  'url': 'https://open.spotify.com/track/6QmQkPEemgfTtX1pjgF0id',
  'preview_url': None},
 {'name': 'might as well might swell bright bell light fell kite dwell fight knell',
  'url': 'https://open.spotify.com/track/5EziInLGuWs7mOxIxS3oyW',
  'preview_url': None},
 {'name': 'nil',
  'url': 'https://open.spotify.com/track/6X8q8PQDUWsKMk5UXAVGel',
  'preview_url': None},
 {'name': 'Sinkin',
  'url': 'https://open.spotify.com/track/75pG923lzUTMDszqvBVZMc',
  'preview_url': None},
 {'name': 'Feels Like..',
  'url': 'https://open.spotify.com/track/5bc5MHu1cRr8Nz870T5rvb',
  'preview_url': None},
 {'name': 'Da Da da',
  'url': 'https://open.spotify.com/track/2SdAAnHTzKei7SHETNhcCw',
  'preview_url': None},
 {'name': 'strapped',
  'url': 'https://open.spotify.com/track/4bOUUJKrtT4IEYB0KpKONt',
  'preview_url': None},
 {'name': 'Dazin',
  'url': 'https://open

Here is a really important function, allowing us to get playlists of a specific country, and propose local music to the tourist.

In [64]:
def get_local_playlists(country, limit=10):
    """
    Récupère les playlists locales populaires pour un pays donné.
    
    :param country: Code du pays (par défaut : 'FR').
    :param limit: Nombre maximum de playlists à retourner.
    """
    try:
        playlists = sp.featured_playlists(country=country, limit=limit)
        return [
            {
                'name': playlist['name'],
                'url': playlist['external_urls']['spotify'],
                'description': playlist['description']
            }
            for playlist in playlists['playlists']['items']
        ]
    except Exception as e:
        print(f"Erreur lors de la récupération des playlists locales : {e}")
        return []

In [65]:
lp = get_local_playlists('FR')
lp

Erreur lors de la récupération des playlists locales : HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)


[]

This function is also really important, because it takes into account the tourist's tastes, in terms of genres of music, artists and favorite music, and sends him personalized recommendations!

In [ ]:
def get_recommendations(seed_artists=None, seed_genres=None, seed_tracks=None, limit=10):
    """
    Récupère des recommandations basées sur des graines (artistes, genres, morceaux).
    
    :param seed_artists: Liste des IDs d'artistes (graines).
    :param seed_genres: Liste des genres musicaux (graines).
    :param seed_tracks: Liste des IDs de morceaux (graines).
    :param limit: Nombre maximum de recommandations.
    """
    try:
        recommendations = sp.recommendations(seed_artists=seed_artists, seed_genres=seed_genres, seed_tracks=seed_tracks, limit=limit)
        return [
            {
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'url': track['external_urls']['spotify'],
                'preview_url': track['preview_url']
            }
            for track in recommendations['tracks']
        ]
    except Exception as e:
        print(f"Erreur lors de la récupération des recommandations : {e}")
        return []

Then this function extract the newest releases in a specific country, allowing the tourist to really appreciate the music listened to in real time by the inhabitants.

In [67]:
def get_new_releases(country, limit=10):
    """
    Récupère les nouvelles sorties musicales pour un pays donné.
    
    :param country: Code du pays (par défaut : 'FR').
    :param limit: Nombre maximum de sorties à retourner.
    """
    try:
        releases = sp.new_releases(country=country, limit=limit)
        return [
            {
                'name': album['name'],
                'artist': album['artists'][0]['name'],
                'url': album['external_urls']['spotify']
            }
            for album in releases['albums']['items']
        ]
    except Exception as e:
        print(f"Erreur lors de la récupération des nouvelles sorties : {e}")
        return []

In [68]:
nr = get_new_releases('FR')
nr

[{'name': 'JVLIVS III : Ad Finem',
  'artist': 'SCH',
  'url': 'https://open.spotify.com/album/6pWHaxvJmknxpUnWh6TcaJ'},
 {'name': 'Inarrêtable',
  'artist': 'Jul',
  'url': 'https://open.spotify.com/album/3txmBSGHr4m3ybYbwV0BPM'},
 {'name': "L'album de la promo 2024",
  'artist': 'Star Academy',
  'url': 'https://open.spotify.com/album/7LXBJT1FsiJaudUCY2rHZQ'},
 {'name': 'II',
  'artist': 'Camille Lellouche',
  'url': 'https://open.spotify.com/album/4bur9otz02TklVA57a7bYO'},
 {'name': 'Hybride',
  'artist': 'DJ Kawest',
  'url': 'https://open.spotify.com/album/0Y1jicwKfOIp4q88Y5n0YK'},
 {'name': 'APOCALYPSE',
  'artist': 'Gazo',
  'url': 'https://open.spotify.com/album/2S3b0g5oagefbTJBf2OVn9'},
 {'name': 'KINTSUGI',
  'artist': 'Dinos',
  'url': 'https://open.spotify.com/album/7o1zpKWhdh1YCvDmPGQ1qt'},
 {'name': 'Je suis là',
  'artist': 'Lara Fabian',
  'url': 'https://open.spotify.com/album/0sRO2vkKqdixkNxIHgf190'},
 {'name': 'Vers les jours heureux',
  'artist': 'Esmée',
  'url': '

And then this gets the tracks of a specific playlist.

In [69]:
def get_playlist_tracks(playlist_id):
    """
    Récupère les morceaux d'une playlist.
    
    :param playlist_id: ID de la playlist.
    """
    try:
        tracks = sp.playlist_tracks(playlist_id)
        return [
            {
                'name': track['track']['name'],
                'artist': track['track']['artists'][0]['name'],
                'url': track['track']['external_urls']['spotify']
            }
            for track in tracks['items']
        ]
    except Exception as e:
        print(f"Erreur lors de la récupération des morceaux de la playlist : {e}")
        return []

In [ ]:
pt = get_playlist_tracks()
pt

## Scraping using BeautifulSoup

We then employ BeatifulSoup, a powerful Python library, for web scraping to efficiently gather and parse data from web pages. This tool is crucial for extracting detailed event information from the 'Shotgun' platform, which lists various cultural and music events. By leveraging BeautifulSoup, we can automate the process of accessing large volumes of data on event times, locations, descriptions, and more, which is essential for analyzing trends and preferences within the entertainment industry. This method enhances our project's capability to handle and analyze data at scale, providing a robust foundation for our insights.

<div>
  <ol>
    <li><strong>City Collection:</strong> Start by scraping a list of cities from the 'Shotgun' platform, which sets the groundwork for deeper analysis.</li>
    <li><strong>Event Detail Function:</strong> Develop a function to extract detailed information about each event for a specified city. This function utilizes web scraping techniques to collect data such as event names, descriptions, locations, and timings from each event's page.</li>
  </ol>
  <p>This structured approach helps in systematically analyzing the entertainment sector's events, city by city, providing a comprehensive understanding of the data gathered.</p>
</div>

Let's import the libraries we need.

In [1]:
import requests
from bs4 import BeautifulSoup
import json

Now let's initiate the url and headers, and parse the HTML content of the website of shotgun to get all the cities.

In [2]:
# URL de base pour les requêtes
base_url = "https://shotgun.live"
# URL de la page des événements de la ville
city_events_url = f"{base_url}/en/cities"

# En-têtes HTTP pour simuler un navigateur
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Requête HTTP pour obtenir la page des événements de la ville
response = requests.get(city_events_url, headers=headers)
response.raise_for_status()  # S'assure que la requête a réussi

# Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link as="image" imagesrcset="/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fgradient-tl.6419c00f.webp&amp;w=750&amp;q=75&amp;dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF 1x, /_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fgradient-tl.6419c00f.webp&amp;w=1920&amp;q=75&amp;dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF 2x" rel="preload"/><link data-precedence="next" href="/_next/static/css/9520e37f74f143c1.css?dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/8a7556e5ee603f9f.css?dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/099dd072b1ef6436.css?dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF" rel="stylesheet"/><link data-precedence="next" href="/_next/static/css/a2a50197af278731.css?dpl=dpl_GSsAnu1XoAA3eejsFzX49Pe5EMCF" rel="stylesheet"/><link data-precedence="next" href=

In [3]:
# Trouve tous les liens d'événements
city_names = [link['href'].split('/')[-1] for link in soup.find_all('a', href=True) if '/en/cities/' in link['href']]
city_names

['bruxelles',
 'aracaju',
 'balneario-camboriu',
 'bauru',
 'belo-horizonte',
 'belem',
 'brasilia',
 'caxias-do-sul',
 'curitiba',
 'florianopolis',
 'fortaleza',
 'garibaldi',
 'goiania',
 'heliodora',
 'ipatinga',
 'joao-pessoa',
 'lages',
 'londrina',
 'manaus',
 'porto-alegre',
 'recife',
 'ribeirao-preto',
 'rio-de-janeiro',
 'salvador',
 'santa-maria',
 'sao-carlos',
 'sao-luis',
 'sao-paulo',
 'taubate',
 'teresina',
 'uberaba',
 'vitoria',
 'sofia',
 'montreal',
 'toronto',
 'copenhagen',
 'aix-marseille',
 'amiens',
 'angers',
 'annecy',
 'arles-avignon',
 'besancon',
 'biarritz',
 'bordeaux',
 'brest',
 'caen',
 'cherbourg',
 'clermont-ferrand',
 'cote-d-azur',
 'dijon',
 'grenoble',
 'la-rochelle',
 'la-reunion',
 'le-havre',
 'le-mans',
 'lille',
 'limoges',
 'lyon',
 'martinique',
 'montpellier',
 'nancy',
 'nantes',
 'orleans',
 'paris',
 'pau',
 'perpignan',
 'poitiers',
 'reims',
 'rennes',
 'rouen',
 'saint-martin',
 'strasbourg',
 'toulouse',
 'tours',
 'vannes',
 'b

After collecting all the cities available in Shotgun, we can crete a function that will get all the events of a specified city (the one where the tourist is going), and for each of these events extract all the detail.

In [90]:
city = "paris"
def get_events(city):
    base_url = "https://shotgun.live"
    city_url = f"{base_url}/fr/cities/{city}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    response = requests.get(city_url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    event_links = [base_url + link['href'] for link in soup.find_all('a', href=True) if '/fr/events/' in link['href']]
    events = []

    for link in event_links:
        event_response = requests.get(link, headers=headers)
        event_response.raise_for_status()
        event_soup = BeautifulSoup(event_response.text, 'html.parser')
        title = event_soup.find('h1').text if event_soup.find('h1') else 'No Title'
        description = event_soup.find("div", class_="whitespace-pre-wrap break-words")
        description_text = description.get_text(strip=True) if description else "No Description"

        json_soup = event_soup.find("div", class_="container")
        script = json_soup.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            json_df = pd.json_normalize(data)  # Normalizing JSON data into a DataFrame
            json_df['Event Name'] = title  # Adding title to the JSON DataFrame
            json_df['Description'] = description_text  # Adding description
            json_df['Link'] = link  # Adding link
            events.append(json_df)
        else:
            events.append(pd.DataFrame({
                'Event Name': [title],
                'Description': [description_text],
                'Link': [link]
            }))

    # Concatenating all dataframes
    events_df = pd.concat(events, ignore_index=True)
    return events_df

# Example of using the function
city_name = "paris"
events_data = get_events(city_name)
events_data

,@context,@type,name,url,image,startDate,doorTime,endDate,eventStatus,eventAttendanceMode,...,location.address.addressCountry,location.geo.@type,location.geo.latitude,location.geo.longitude,organizer.@type,organizer.name,organizer.url,Event Name,Description,Link
0,https://schema.org,MusicEvent,Mutant. X Magasins Généraux,https://shotgun.live/fr/events/mutant-x-magasi...,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-14T15:00:00.000Z,2024-12-14T15:00:00.000Z,2024-12-15T05:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.894699,2.416241,Organization,MUTANT.,https://shotgun.live/fr/venues/mutant,Mutant. X Magasins Généraux,MUTANT. EXTENDED editionOn revient le samedi 1...,https://shotgun.live/fr/events/mutant-x-magasi...
1,https://schema.org,MusicEvent,Rex Club Presents: Bakermat & Saverio,https://shotgun.live/fr/events/rex-club-presen...,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T22:59:00.000Z,2024-12-12T22:59:00.000Z,2024-12-13T06:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.870573,2.347382,Organization,REX CLUB,https://shotgun.live/fr/venues/rex-club,Rex Club Presents: Bakermat & Saverio,Rex Club Presents: Bakermat & Saverio,https://shotgun.live/fr/events/rex-club-presen...
2,https://schema.org,MusicEvent,Jedismieux X Hyperbrat,https://shotgun.live/fr/events/copy-of-je-di-s...,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T22:00:00.000Z,2024-12-12T22:00:00.000Z,2024-12-13T05:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.858615,2.369167,Organization,RAW’corp,https://shotgun.live/fr/venues/raw-corp,Jedismieux X Hyperbrat,SPECIAL EDITION 🖤JEDISMIEUX X HYPERBRAT : HYPE...,https://shotgun.live/fr/events/copy-of-je-di-s...
3,https://schema.org,MusicEvent,"Faul&Wad, Mont Rouge, Maesic, James Mac, Soube...",https://shotgun.live/fr/events/aaaxkomma,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T22:30:00.000Z,2024-12-12T22:30:00.000Z,2024-12-13T04:30:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.879287,2.282754,Organization,KOMMA,https://shotgun.live/fr/venues/komma,"Faul&Wad, Mont Rouge, Maesic, James Mac, Soube...",All Around AgencyLINE UP :MONT ROUGEMELITTAH B...,https://shotgun.live/fr/events/aaaxkomma
4,https://schema.org,MusicEvent,"Casa Ata W/ Sara Siu, Luca Bree, Raph & Gauth",https://shotgun.live/fr/events/casa-ata-w-sara...,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T22:00:00.000Z,2024-12-12T22:00:00.000Z,2024-12-13T05:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.868952,2.343454,Organization,Silencio Club,https://shotgun.live/fr/venues/silencio-club,"Casa Ata W/ Sara Siu, Luca Bree, Raph & Gauth",CASA ATASARA SIULUCA BREERAPH & GAUTH,https://shotgun.live/fr/events/casa-ata-w-sara...
5,https://schema.org,MusicEvent,Apero 90's-00's - Jeudi 12 Décembre,https://shotgun.live/fr/events/event-3949,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T21:00:00.000Z,2024-12-12T21:00:00.000Z,2024-12-13T04:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.869067,2.343365,Organization,SACRÉ,https://shotgun.live/fr/venues/sacre,Apero 90's-00's - Jeudi 12 Décembre,Rendez-vous tous les jeudis au Sacré.22h00 - 0...,https://shotgun.live/fr/events/event-3949
6,https://schema.org,MusicEvent,Soirée Stand-Up À La Flèche D'or !,https://shotgun.live/fr/events/stand-up-fleche...,https://res.cloudinary.com/shotgun/image/uploa...,2024-12-12T19:00:00.000Z,2024-12-12T19:00:00.000Z,2024-12-12T22:59:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,FR,GeoCoordinates,48.859433,2.402906,Organization,La Flèche d'Or,https://shotgun.liv

As you can see, we get only 14 events for Paris. Isn't that little ? We thought that it was, and that there was surely a problem. That's when I noticed that shotgun only displayed 14 events on its page, and that you had to click on a button to get 14 more, and again to get 14 more, and so on. Knowing that in Paris, there are currently about 1250 events... So let's make a loop! But why doesn't it work with BeatifulSoup ?

The use of Selenium became necessary because BeautifulSoup, while excellent for parsing and extracting data from static HTML, does not support interactions with web elements such as clicking buttons. Some events on the 'Shotgun' platform only appear after interacting with a "View More" button, which dynamically loads additional content. Since BeautifulSoup cannot handle these dynamic elements, Selenium was employed to automate browser interactions, allowing the script to simulate clicks and retrieve all available event data dynamically loaded on the page.

Here is the new function !

## Scraping mixing BeautifulSoup and Selenium 

In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time

In [52]:
def get_all_html(city):
    base_url = "https://shotgun.live"
    city_url = f"{base_url}/fr/cities/{city}"

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Open the page
    driver.get('https://shotgun.live/fr/cities/paris')

    try:
        i=0
        while True:
            try:
                # Use JavaScript to execute click directly
                button = driver.find_element(By.CSS_SELECTOR, "button.text-sm.tracking-wider.h-10.rounded.px-4.py-2.inline-flex.items-center.justify-center.whitespace-nowrap.font-bold.uppercase.text-foreground")
                driver.execute_script("arguments[0].click();", button)
                time.sleep(2)  # Wait for the page to load more items
                i=i+1
            except NoSuchElementException:
                break  # Stop if the button no longer exists
    except ElementClickInterceptedException:
        # If something blocks the button, this could handle it, e.g., closing a popup
        driver.find_element(By.CSS_SELECTOR, "div.CybotCookiebotDialogBodyButton").click()  # Example selector for a close button on a popup
        time.sleep(1)  # Short pause after closing the popup
        driver.execute_script("arguments[0].click();", button)
            # Get the page source after all interactions
    html = driver.page_source
    driver.quit()
    return html

In [62]:
def get_events(html):    
# Now use BeautifulSoup to parse the loaded page
    soup = BeautifulSoup(html, 'html.parser')
    event_links = [base_url + link['href'] for link in soup.find_all('a', href=True) if '/fr/events/' in link['href']]
    events = []

    for link in event_links:
        event_response = requests.get(link)
        event_soup = BeautifulSoup(event_response.text, 'html.parser')
        try: 
            moods = event_soup.find("div", class_="flex flex-wrap gap-2").find_all("a")
            moods_string = ",".join(mood.get_text(strip=True) for mood in moods)
        except:
            moods_string=""


        json_soup = event_soup.find("div", class_="container")
        script = json_soup.find('script', type='application/ld+json')
        if script:
            data = json.loads(script.string)
            json_df = pd.json_normalize(data)
            json_df['moods'] = moods_string
            events.append(json_df)
            
    events_df = pd.concat(events, ignore_index=True)
    return events_df

In [55]:
# Example of using the function
city_name = "paris"
html = get_all_html(city_name)

In [63]:
events_data = get_events(html)
events_data

,@context,@type,name,url,image,startDate,doorTime,endDate,eventStatus,eventAttendanceMode,...,location.address.addressLocality,location.address.postalCode,location.address.addressCountry,location.geo.@type,location.geo.latitude,location.geo.longitude,organizer.@type,organizer.name,organizer.url,moods
0,https://schema.org,MusicEvent,Hypershot — Jespfur (+) Bby Eco (+) Izzy Camin...,https://shotgun.live/fr/events/hypershot-jespf...,https://res.cloudinary.com/shotgun/image/uploa...,2025-02-06T18:30:00.000Z,2025-02-06T18:30:00.000Z,2025-02-06T22:30:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75011,FR,GeoCoordinates,48.853528,2.375755,Organization,BADABOUM,https://shotgun.live/fr/venues/badaboum-club,"pop,electro"
1,https://schema.org,MusicEvent,"Organïk Hard Psy Edition: La Penderie Noire, M...",https://shotgun.live/fr/events/organik-hard-ps...,https://res.cloudinary.com/shotgun/image/uploa...,2025-01-09T22:59:00.000Z,2025-01-09T22:59:00.000Z,2025-01-10T06:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75002,FR,GeoCoordinates,48.870573,2.347382,Organization,REX CLUB,https://shotgun.live/fr/venues/rex-club,"electro,hard techno,techno,psytrance"
2,https://schema.org,MusicEvent,Complet - Jolagreen23 - La Cigale,https://shotgun.live/fr/events/jolagreen-23-la...,https://res.cloudinary.com/shotgun/image/uploa...,2025-01-09T19:00:00.000Z,2025-01-09T19:00:00.000Z,2025-01-09T22:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75018,FR,GeoCoordinates,48.882519,2.340163,Organization,PEDRO BOOKING,https://shotgun.live/fr/venues/pedro-booking,rap
3,https://schema.org,MusicEvent,Jukebox 09/01,https://shotgun.live/fr/events/jukebox-09-01,https://res.cloudinary.com/shotgun/image/uploa...,2025-01-09T22:45:00.000Z,2025-01-09T22:45:00.000Z,2025-01-10T04:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75008,FR,GeoCoordinates,48.870700,2.309207,Organization,Stargazing Group,https://shotgun.live/fr/venues/lalaland,
4,https://schema.org,MusicEvent,Malaa,https://shotgun.live/fr/events/malaa,https://res.cloudinary.com/shotgun/image/uploa...,2025-01-09T22:00:00.000Z,2025-01-09T22:00:00.000Z,2025-01-10T04:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75013,FR,GeoCoordinates,48.841110,2.369852,Organization,34 QUAI D'AUSTERLITZ,https://shotgun.live/fr/venues/34-quai-d-auste...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,https://schema.org,MusicEvent,Pogo Car Crash Control - La Cigale - Paris,https://shotgun.live/fr/events/pogo-car-crash-...,https://res.cloudinary.com/shotgun/image/uploa...,2025-11-27T18:30:00.000Z,2025-11-27T18:30:00.000Z,2025-11-27T21:30:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75018,FR,GeoCoordinates,48.882519,2.340163,Organization,W Live,https://shotgun.live/fr/venues/w-live,"punk,rock,metal"
1407,https://schema.org,MusicEvent,Vendredi Sur Mer - La Cigale - Paris,https://shotgun.live/fr/events/vendredi-sur-me...,https://res.cloudinary.com/shotgun/image/uploa...,2025-12-02T18:30:00.000Z,2025-12-02T18:30:00.000Z,2025-12-02T21:30:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75018,FR,GeoCoordinates,48.882519,2.340163,Organization,W Live,https://shotgun.live/fr/venues/w-live,pop
1408,https://schema.org,MusicEvent,Les Yeux Dla Tete,https://shotgun.live/fr/events/les-yeux-dla-tete,https://res.cloudinary.com/shotgun/image/uploa...,2025-12-05T18:30:00.000Z,2025-12-05T18:30:00.000Z,2025-12-05T22:00:00.000Z,https://schema.org/EventScheduled,https://schema.org/OfflineEventAttendanceMode,...,Paris,75018,FR,GeoCoordinates,48.882519,2.340163,Organization,Ovastand,https://shotgun.live/fr/venues/ovastand,pop
1409,https://schema.org,Musi

In [15]:
events_data.columns

Index(['@context', '@type', 'name', 'url', 'image', 'startDate', 'doorTime',
       'endDate', 'eventStatus', 'eventAttendanceMode', 'description',
       'performer', 'offers', 'location.@type', 'location.name',
       'location.address.@type', 'location.address.streetAddress',
       'location.address.addressLocality', 'location.address.postalCode',
       'location.address.addressCountry', 'location.geo.@type',
       'location.geo.latitude', 'location.geo.longitude', 'organizer.@type',
       'organizer.name', 'organizer.url', 'Event Name', 'Description', 'Link'],
      dtype='object')

In [65]:
events_data.to_excel(f"Events_{city_name}.xlsx", index=False)

And here it is, a detailed DataFrame of 1235 events !

You can see below the test of the click button loop with Selenium.

In [108]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

# Setup Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the page
driver.get('https://shotgun.live/fr/cities/paris')
time.sleep(2)  # Allow the page to load

try:
    while True:
        try:
            # Use JavaScript to execute click directly
            button = driver.find_element(By.CSS_SELECTOR, "button.text-sm.tracking-wider.h-10.rounded.px-4.py-2.inline-flex.items-center.justify-center.whitespace-nowrap.font-bold.uppercase.text-foreground")
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)  # Wait for the page to load more items
        except NoSuchElementException:
            break  # Stop if the button no longer exists
except ElementClickInterceptedException:
    # If something blocks the button, this could handle it, e.g., closing a popup
    driver.find_element(By.CSS_SELECTOR, "div.CybotCookiebotDialogBodyButton").click()  # Example selector for a close button on a popup
    time.sleep(1)  # Short pause after closing the popup
    driver.execute_script("arguments[0].click();", button)

# Close the driver
driver.quit()

The data we're gathering on events through web scraping is invaluable for various aspects of our project, offering comprehensive insights that can be leveraged in multiple ways:

* Event Details: Names, dates, and times provide basic but essential information for scheduling and planning.
* Location Information: Addresses and geographic coordinates are crucial for mapping services and logistical arrangements.
* Organizer Details: Knowing the organizer and their online presence can help in assessing the credibility and reach of the event.

And finally, the maine one is :
* Description: Rich descriptions are ripe for NLP to extract themes, key terms, and sentiment, offering a deeper understanding of what the event offers and appealing to specific audience interests.
This structured data can enhance user experience by enabling targeted content delivery, personalized recommendations, and optimized search functionalities within your platform.